# **P1. Recogida datos no estructurados. Reviews**

**Enlace** al dataset de reviews en Kaggle: https://www.kaggle.com/datasets/jaidalmotra/movies-review 

In [1]:
!pip install elasticsearch --quiet

In [2]:
!pip install kagglehub

In [3]:
import json
from elasticsearch import Elasticsearch
es = Elasticsearch ("http://elasticsearch:9200")

## Captura de reseñas

In [4]:
import kagglehub
import os
import pandas as pd

# Download latest version
path = kagglehub.dataset_download("jaidalmotra/movies-review")

print("Path to dataset files:", path)

# List all files in the directory
path = os.path.join(path, os.listdir(path)[0])

# Create a dictionary to store DataFrames for each genre
genre_dfs = {}

# Read all CSV files
for file_name in os.listdir(path):
    df = pd.read_csv(os.path.join(path, file_name))

    # Extract the genre from the file name
    genre = file_name.split('.')[0]

    # Store the DataFrame in the dictionary
    genre_dfs[genre] = df

Path to dataset files: /home/jovyan/.cache/kagglehub/datasets/jaidalmotra/movies-review/versions/1


In [5]:
genre_dfs['Action']

,name,year,movie_rated,run_length,genres,release_date,rating,num_raters,num_reviews,review_url
0,The Dark Knight,2008,PG-13,2h 32min,Action; Crime; Drama;,18 July 2008 (USA),9.0,2224522,6836,https://www.imdb.com/title/tt0468569/reviews/_...
1,Inception,2010,PG-13,2h 28min,Action; Adventure; Sci-Fi;,16 July 2010 (USA),8.8,1981675,3820,https://www.imdb.com/title/tt1375666/reviews/_...
2,The Matrix,1999,R,2h 16min,Action; Sci-Fi;,31 March 1999 (USA),8.7,1619761,4281,https://www.imdb.com/title/tt0133093/reviews/_...
3,The Lord of the Rings: The Fellowship of the Ring,2001,PG-13,2h 58min,Action; Adventure; Drama;,19 December 2001 (USA),8.8,1609165,5365,https://www.imdb.com/title/tt0120737/reviews/_...
4,The Dark Knight Rises,2012,PG-13,2h 44min,Action; Adventure;,20 July 2012 (USA),8.4,1470329,2979,https://www.imdb.com/title/tt1345836/reviews/_...
...,...,...,...,...,...,...,...,...,...,...
95,Harry Potter and the Order of the Phoenix,2007,PG-13,2h 18min,Action; Adventure;,11 July 2007 (USA),7.5,489759,1189,https://www.imdb.com/title/tt0373889/reviews/_...
96,Spider-Man: Homecoming,2017,PG-13,2h 13min,Action; Adventure; Sci-Fi;,7 July 2017 (USA),7.4,488794,1356,https://www.imdb.com/title/tt2250912/reviews/_...
97,Rise of the Planet of the Apes,2011,PG-13,1h 45min,Action; Drama; Sci-Fi;,5 August 2011 (USA),7.6,488748,706,https://www.imdb.com/title/tt1318514/reviews/_...
98,"I, Robot",2004,PG-13,1h 55min,Action; Drama; Sci-Fi;,16 July 2004 (USA),7.1,488324,839,https://www.imdb.com/title/tt0343818/reviews/_...


In [6]:
# Create a dictionary to store review_urls for each genre
reviews_dict = {}

# Extract review_url column from each genre DataFrame
for genre, df in genre_dfs.items():
    if 'review_url' in df.columns:
        reviews_dict[genre] = df['review_url'].values

# Find the maximum length among all review_url lists
max_length = max(len(urls) for urls in reviews_dict.values())

# Pad shorter lists with None to make them equal length
for genre in reviews_dict:
    current_length = len(reviews_dict[genre])
    if current_length < max_length:
        reviews_dict[genre] = np.append(reviews_dict[genre], [None] * (max_length - current_length))

# Create the new DataFrame
reviews_df = pd.DataFrame(reviews_dict)

# Display number of non-null values in each column
print("\nNumber of reviews per genre:")
print(reviews_df.count())


Number of reviews per genre:
Action       100
Crime        100
Mystery      100
Music        100
Drama        100
Biography    100
Thriller     100
War          100
History      100
Sci-Fi       100
Horror       100
Romance      100
Adventure    100
Animation    100
Fantasy      100
Sport        100
Comedy       100
dtype: int64


In [7]:
reviews_df.head(10)

,Action,Crime,Mystery,Music,Drama,Biography,Thriller,War,History,Sci-Fi,Horror,Romance,Adventure,Animation,Fantasy,Sport,Comedy
0,https://www.imdb.com/title/tt0468569/reviews/_...,https://www.imdb.com/title/tt0468569/reviews/_...,https://www.imdb.com/title/tt0114369/reviews/_...,https://www.imdb.com/title/tt0253474/reviews/_...,https://www.imdb.com/title/tt0111161/reviews/_...,https://www.imdb.com/title/tt0108052/reviews/_...,https://www.imdb.com/title/tt0102926/reviews/_...,https://www.imdb.com/title/tt0361748/reviews/_...,https://www.imdb.com/title/tt0108052/reviews/_...,https://www.imdb.com/title/tt1375666/reviews/_...,https://www.imdb.com/title/tt0081505/reviews/_...,https://www.imdb.com/title/tt0109830/reviews/_...,https://www.imdb.com/title/tt1375666/reviews/_...,https://www.imdb.com/title/tt0910970/reviews/_...,https://www.imdb.com/title/tt0167260/reviews/_...,https://www.imdb.com/title/tt0118715/reviews/_...,https://www.imdb.com/title/tt2015381/reviews/_...
1,https://www.imdb.com/title/tt1375666/reviews/_...,https://www.imdb.com/title/tt0110912/reviews/_...,https://www.imdb.com/title/tt0482571/reviews/_...,https://www.imdb.com/title/tt2582802/reviews/_...,https://www.imdb.com/title/tt0468569/reviews/_...,https://www.imdb.com/title/tt0993846/reviews/_...,https://www.imdb.com/title/tt0407887/reviews/_...,https://www.imdb.com/title/tt0120815/reviews/_...,https://www.imdb.com/title/tt0112573/reviews/_...,https://www.imdb.com/title/tt0133093/reviews/_...,https://www.imdb.com/title/tt0078748/reviews/_...,https://www.imdb.com/title/tt0120338/reviews/_...,https://www.imdb.com/title/tt0120737/reviews/_...,https://www.imdb.com/title/tt0266543/reviews/_...,https://www.imdb.com/title/tt0167261/reviews/_...,https://www.imdb.com/title/tt0405159/reviews/_...,https://www.imdb.com/title/tt0088763/reviews/_...
2,https://www.imdb.com/title/tt0133093/reviews/_...,https://www.imdb.com/title/tt0068646/reviews/_...,https://www.imdb.com/title/tt0209144/reviews/_...,https://www.imdb.com/title/tt3783958/reviews/_...,https://www.imdb.com/title/tt0137523/reviews/_...,https://www.imdb.com/title/tt0099685/reviews/_...,https://www.imdb.com/title/tt0209144/reviews/_...,https://www.imdb.com/title/tt0093058/reviews/_...,https://www.imdb.com/title/tt1504320/reviews/_...,https://www.imdb.com/title/tt0816692/reviews/_...,https://www.imdb.com/title/tt0816711/reviews/_...,https://www.imdb.com/title/tt0338013/reviews/_...,https://www.imdb.com/title/tt0167260/reviews/_...,https://www.imdb.com/title/tt0110357/reviews/_...,https://www.imdb.com/title/tt0076759/reviews/_...,https://www.imdb.com/title/tt0075148/reviews/_...,https://www.imdb.com/title/tt0266543/reviews/_...
3,https://www.imdb.com/title/tt0120737/reviews/_...,https://www.imdb.com/title/tt0114369/reviews/_...,https://www.imdb.com/title/tt1130884/reviews/_...,https://www.imdb.com/title/tt1727824/reviews/_...,https://www.imdb.com/title/tt0110912/reviews/_...,https://www.imdb.com/title/tt0112573/reviews/_...,https://www.imdb.com/title/tt1130884/reviews/_...,https://www.imdb.com/title/tt0457430/reviews/_...,https://www.imdb.com/title/tt2024544/reviews/_...,https://www.imdb.com/title/tt0848228/reviews/_...,https://www.imdb.com/title/tt0054215/reviews/_...,https://www.imdb.com/title/tt0119217/reviews/_...,https://www.imdb.com/title/tt1345836/reviews/_...,https://www.imdb.com/title/tt1049413/reviews/_...,https://www.imdb.com/title/tt0080684/reviews/_...,https://www.imdb.com/title/tt1291584/reviews/_...,https://www.imdb.com/title/tt1049413/reviews/_...
4,https://www.imdb.com/title/tt1345836/reviews/_...,https://www.imdb.com/title/tt0102926/reviews/_...,https://www.imdb.com/title/tt0114814/reviews/_...,https://www.imdb.com/title/tt1517451/reviews/_...,https://www.imdb.com/title/tt0109830/reviews/_...,https://www.imdb.com/title/tt0268978/reviews/_...,https://www.imdb.com/title/tt0266697/reviews/_...,https://www.imdb.com/title/tt0078788/reviews/_...,https://www.imdb.com/title/tt5013056/reviews/_...,https://www.

In [8]:
import requests
from bs4 import BeautifulSoup

def extract_imdb_reviews(url):
    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all review containers
        review_containers = soup.find_all('div', class_='review-container')

        reviews = []
        for container in review_containers:
            # Extract the review text
            review_text = container.find('div', class_='text show-more__control').text.strip()
            reviews.append(review_text)

        return reviews
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return []

cleaned_df = pd.DataFrame(columns=['review_text', 'title'])
    
# URL of the IMDB reviews page
for genre in reviews_df.columns:
    
    print(genre)
    
    reviews_genre = reviews_df[genre]

    genre_movies = genre_dfs[genre]
    
    for idx, url in enumerate(reviews_genre):
        
        if idx < len(genre_movies):
            # Obtener el nombre de la película del DataFrame de género
            title = genre_movies['name'].iloc[idx]

            # Extraer las reseñas utilizando el enlace de reviews
            reviews = extract_imdb_reviews(url)  # Esta función debe devolver una lista de reseñas

            # Asegúrate de que reviews no esté vacío
            if reviews:
                # Crear un DataFrame temporal para las reseñas
                temp_df = pd.DataFrame({
                    'title': title,
                    'review_text': reviews  # Aquí la lista de reseñas se convierte en una columna
                })

                # Repetir el título para cada reseña
                temp_df = temp_df.explode('review_text')

                # Concatenar el DataFrame temporal con el DataFrame limpio
                cleaned_df = pd.concat([cleaned_df, temp_df], ignore_index=True)       
                                                                              
        
    #break

Action
Crime
Mystery
Music
Drama
Biography
Thriller
War
History
Sci-Fi
Horror
Romance
Adventure
Animation
Fantasy
Sport
Comedy


In [9]:
cleaned_df.head(30)

,review_text,title
0,Best movie ever. Heath ledger's work is phenom...,The Dark Knight
1,This movie is a work of art. The finest sequel...,The Dark Knight
2,It is just what you want for the best movie. G...,The Dark Knight
3,"Confidently directed, dark, brooding, and pack...",The Dark Knight
4,I got to see The Dark Knight on Wednesday nigh...,The Dark Knight
5,We've been subjected to enormous amounts of hy...,The Dark Knight
6,"I couldn't believe ""The Dark knight"" could liv...",The Dark Knight
7,"Dark, yes, complex, ambitious. Christopher Nol...",The Dark Knight
8,I think that I could wax lyrical but this movi...,The Dark Knight
9,Im just gonna start off by saying I LOVE this ...,The Dark Knight


In [10]:
cleaned_df.to_csv("./cleaned_df.csv")

In [11]:
cleaned_df = pd.read_csv("./cleaned_df.csv")

## Inserción en ElasticSearch

In [14]:
if not es.indices.exists(index="reviews"):
    es.indices.create(index="reviews")
    print("Índice reviews creado.")
else:
    print("El índice reviews ya existe.")


Índice reviews creado.


In [15]:
### Normalizacion

import re
def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

cleaned_df['review_text'] = cleaned_df['review_text'].apply(clean_text)

In [16]:
from elasticsearch.helpers import bulk

docs = []
columns = cleaned_df.columns
for idx, row in cleaned_df.iterrows():
    tupla = {'_index': 'reviews', '_id': idx, '_source': {columns[0]: row[0], columns[1]: row[1]}}
    
    docs.append(tupla)

docs[1]

{'_index': 'reviews',
 '_id': 1,
 '_source': {'Unnamed: 0': 1,
  'review_text': 'this movie is a work of art the finest sequel ever made i dont think we will see another movie like this for a long time heath ledgers joker is the best movie charachter i have ever seen by far avengers endgame is great but the dark knight is much better the best batman ever the best joker ever the best dc movie ever the best superhero movie ever ando for me the best movie ever'}}

In [17]:
# Index the documents in bulk

res = bulk(es, docs)
print(res)

(42407, [])


In [18]:
res = es.search(index='reviews', body={
    'query': {
    'match': { 'review_text': 'inception' }
    }
})
print(json.dumps(res.body, indent=4))

{
    "took": 1386,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 218,
            "relation": "eq"
        },
        "max_score": 10.265317,
        "hits": [
            {
                "_index": "reviews",
                "_type": "_doc",
                "_id": "37",
                "_score": 10.265317,
                "_ignored": [
                    "review_text.keyword"
                ],
                "_source": {
                    "Unnamed: 0": 37,
                    "review_text": "inception is my number one favorite movie off my twelve years on earth its absolutely impossible to explain the incredible that inception has inception is sooooooooo dang exciting edge of seat mind blower and i wish there were more movies like it and more christopher nolans out there congratulations to the well deserved best director writer and produ